In [1]:
import random
import gym
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


from mlagents_envs.environment import UnityEnvironment
from gym_unity.envs import UnityToGymWrapper


c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py:15: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\framework\dtypes.py:601: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\framework\dtypes.py:609: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you s

In [2]:
ENV_NAME = "Environment.exe"

GAMMA = 0.9
LEARNING_RATE = 0.0001

MEMORY_SIZE = 100000
BATCH_SIZE = 32

EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.09
EXPLORATION_DECAY = 0.00009

In [3]:
class DQNSolver:

    def __init__(self, observation_space, action_space):
        self.exploration_rate = EXPLORATION_MAX

        self.action_space = action_space
        self.memory = deque(maxlen=MEMORY_SIZE)

        self.model = Sequential()
        self.model.add(Dense(24, input_shape=(observation_space,), activation="relu"))
        self.model.add(Dense(24, activation="relu"))
        self.model.add(Dense(self.action_space, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(learning_rate=LEARNING_RATE))

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() < self.exploration_rate:
            return random.randrange(self.action_space)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def experience_replay(self):
        if len(self.memory) < BATCH_SIZE:
            return
        batch = random.sample(self.memory, BATCH_SIZE)
        for state, action, reward, state_next, terminal in batch:
            q_update = reward
            if not terminal:
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))
            q_values = self.model.predict(state)
            q_values[0][action] = q_update
            self.model.fit(state, q_values, verbose=0)
        self.exploration_rate *= EXPLORATION_DECAY
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)
    
    def getWeights(self):
        return self.model.get_weights()
    
    def setWeights(self, weights):
        self.model.set_weights(weights)
    
    
    def save(self, name): 
        self.model.save_weights(name)

In [4]:
def classicalRL():
    try:
      env.close()
    except:
      pass
    env = UnityEnvironment(file_name=ENV_NAME, seed=1, side_channels=[])
    env = UnityToGymWrapper(env)
    print("Environment: ", env)
    observation_space = env.observation_space.shape[0]
    action_space = int(env.action_space.nvec)
    dqn_solver = DQNSolver(observation_space, action_space)
    dqn_solverTarget = DQNSolver(observation_space, action_space)
    run = 0
    steps_per_update = 5 # Train the model every x steps
    rewards_summary = []
    while True:
        run += 1
        state = env.reset()
        state = np.reshape(state, [1, observation_space])
        avg_reward = 0
        step_count = 0
        while True:
            step_count += 1
            action = dqn_solver.act(state)
            state_next, reward, terminal, info = env.step(action)
            avg_reward += int(reward)
            state_next = np.reshape(state_next, [1, observation_space])
            dqn_solver.remember(state, action, int(reward), state_next, terminal)
            state = state_next
            if step_count % steps_per_update == 0:
                dqn_solver.experience_replay()
            if terminal:
                dqn_solverTarget.setWeights(dqn_solver.getWeights());
                break
        rewards_summary.append(avg_reward)
        if(run % 10 == 0):
            last_10 = rewards_summary[-10:]   # slice the last 10 elements of the list
            average = sum(last_10) / len(last_10)   # calculate the average of the last 10 elements
            dqn_solverTarget.setWeights(dqn_solver.getWeights())
            print("Run: " + str(run) + ", score: " + str(average))
            if (average >= 9.0):
                    break

In [ ]:
classicalRL()

2023-04-11 12:10:58 INFO [environment.py:498] Environment shut down with return code 3221225781.
